# 📘 Comic Spoiler Detection Demo (Colab Notebook)

## 🛠️ STEP 1: Install Required Packages


In [ ]:
!pip install transformers ultralytics xgboost deepface scikit-learn opencv-python-headless pytesseract


## 📁 STEP 2: Clone the GitHub Repository


In [ ]:
!git clone https://github.com/djcode0718/ComicSpoilerDetection.git
%cd ComicSpoilerDetection

## 📦 STEP 3: Load Models

In [ ]:
import cv2
import numpy as np
import pandas as pd
import pytesseract
from transformers import pipeline
from ultralytics import YOLO
import joblib
from xgboost import XGBClassifier
from deepface.DeepFace import represent
from sklearn.cluster import DBSCAN
from scipy.spatial.distance import cosine


# Set Tesseract path for Colab (Skip if using hosted runtime)
!apt install tesseract-ocr -y
pytesseract.pytesseract.tesseract_cmd = "/usr/bin/tesseract"

# Load HuggingFace Pipelines
caption_generator = pipeline("summarization", model="facebook/bart-large-cnn")
genre_classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Load local models (if uploaded to Colab)
character_detector = YOLO("models/yolov8_comic_characters_detect.pt")
xgboost_model = joblib.load("models/xgboost_spoiler_classifier.pkl")
tfidf_vectorizer = joblib.load("models/tfidf_vectorizer.pkl")
genre_encoder = joblib.load("models/genre_encoder.pkl")

# Define genre labels
genre_labels = ["Sports", "Crime", "Action", "Fantasy", "Sci-Fi", "Romance", "Horror", "Comedy", "Drama", "Mystery", "Superhero"]


## Helper functions


In [ ]:
# 🔹 Feature Extraction Functions
def extract_text_from_image(image_path):
    """Extract text from an image using Tesseract OCR."""
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Could not load image {image_path}")
        return ""

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    text = pytesseract.image_to_string(gray)
    return text.strip()

def generate_caption(text):
    """Generate a short caption using BART summarization."""
    if not text or text.strip() == "":
        return "No caption available"

    try:
        summary = caption_generator(text, max_length=50, min_length=10, do_sample=False)
        return summary[0]['summary_text']
    except Exception as e:
        print(f"Error summarizing text: {e}")
        return "Error generating caption"

def predict_genre(text):
    """Predict comic genre using BART-based zero-shot classification."""
    if not text.strip():
        return "Unknown"

    result = genre_classifier(text, genre_labels)
    predicted_genre = result["labels"][0]

    # Ensure predicted genre is within allowed genres
    if predicted_genre not in genre_labels:
        predicted_genre = "Unknown"

    return predicted_genre

def get_face_embedding(face):
    """Extract face embeddings using DeepFace."""
    try:
        face_embedding = represent(face, model_name="Facenet", enforce_detection=False)
        return face_embedding[0]['embedding']
    except Exception as e:
        print(f"Error extracting face embedding: {e}")
        return None

def detect_unique_characters(image_path):
    """Detect unique characters in a comic page image."""
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Could not read image {image_path}")
        return 0

    results = character_detector(image)
    face_embeddings = []

    # Extract detected characters
    for result in results:
        for box in result.boxes.xyxy:
            x1, y1, x2, y2 = map(int, box)
            face = image[y1:y2, x1:x2]
            if face.size == 0:
                continue

            embedding = get_face_embedding(face)
            if embedding is not None:
                face_embeddings.append(embedding)

    if not face_embeddings:
        print("Unique characters detected: 0")
        return 0

    # Cluster face embeddings to identify unique characters
    clustering = DBSCAN(metric=cosine, eps=0.5, min_samples=1)
    labels = clustering.fit_predict(face_embeddings)

    unique_characters = len(set(labels))
    print(f"Unique characters detected: {unique_characters}")
    return unique_characters

# 🔹 Spoiler Prediction Function
def predict_spoiler(image_path):
    """Predict whether a comic panel contains a spoiler."""
    print(f"Processing image: {image_path}...")

    # Step 1: Extract text
    extracted_text = extract_text_from_image(image_path)
    print(f"Extracted Text: {extracted_text}")

    # Step 2: Generate Caption
    caption = generate_caption(extracted_text)
    print(f"Generated Caption: {caption}")

    # Step 3: Predict Genre
    genre = predict_genre(extracted_text)
    print(f"Predicted Genre: {genre}")

    # Step 4: Count Unique Characters
    unique_character_count = detect_unique_characters(image_path)
    print(f"Unique Character Count: {unique_character_count}")

    # Step 5: Encode Genre
    if genre in genre_encoder.classes_:
        genre_encoded = genre_encoder.transform([genre])[0]
    else:
        genre_encoded = -1  # Assign a default value for unknown genres

    # Step 6: Convert Caption to TF-IDF Features
    caption_features = tfidf_vectorizer.transform([caption]).toarray()  # Shape (1, 500)

    # Step 7: Fix Dimension Mismatch
    numeric_features = np.array([[unique_character_count, genre_encoded]])  # Shape (1, 2)
    X_input = np.hstack((numeric_features, caption_features))  # Shape (1, 502)

    # Step 8: Ensure Correct Shape Before Prediction
    X_input = np.array(X_input).reshape(1, -1)  # Ensure 2D array shape (1, 502)

    # Step 9: Predict Spoiler
    prediction = xgboost_model.predict(X_input)[0]

    # Step 10: Interpret Prediction
    label_mapping = {0: "Unknown", 1: "🟩 Non-Spoiler", 2: "🟥 Spoiler"}
    print(label_mapping.get(prediction, "Unknown"))

## 🖼️ STEP 4: View a Test Image

In [ ]:
import matplotlib.pyplot as plt
img_path = "images/0adacd53-c788-470a-8c38-b68449d504a9.jpg"

img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(6,6))
plt.imshow(img)
plt.title("Test Comic Panel")
plt.axis("off")
plt.show()

## 🧠 STEP 5: Predict the spoiler

In [ ]:
predict_spoiler(img_path)



In [ ]:
import matplotlib.pyplot as plt
img_path = "images/002a6441-5424-4f66-8998-d74a052b92ec (1).jpg"

img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(6,6))
plt.imshow(img)
plt.title("Test Comic Panel")
plt.axis("off")
plt.show()

In [ ]:
predict_spoiler(img_path)